### 메시지 세이버(Message Saver)를 활용한 상태 관리

#### 1. LangGraph로 나만의 챗봇 만들기

In [18]:
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition

# 1. 도시별 관광명소 도구
@tool
def get_attractions(city: str) -> list[str]:
    """
    주어진 도시의 인기 관광명소 5곳을 반환합니다.
    """
    sample = {
        "파리": ["에펠탑", "루브르박물관", "노트르담 대성당", "몽마르트 언덕", "개선문"],
        "도쿄": ["도쿄타워", "센소지", "시부야 스크램블", "신주쿠 교엔", "아키하바라"],
        "뉴욕": ["타임스퀘어", "자유의 여신상", "센트럴파크", "엠파이어 스테이트 빌딩", "브로드웨이"],
    }
    return sample.get(city, ["정보가 없습니다"])

# 2. 항공권 가격 도구
@tool
def get_flight_price(depart: str, arrive: str, date: str) -> str:
    """
    출발지→도착지, 날짜별 예상 항공권 가격을 반환합니다.
    """
    # 주요 도시 간 기본 가격 (원화)
    prices = {
        ("서울", "도쿄"): 350000,
        ("서울", "파리"): 1200000,
        ("서울", "뉴욕"): 1500000,
        ("도쿄", "서울"): 350000,
        ("도쿄", "파리"): 1100000,
        ("파리", "서울"): 1200000,
        ("파리", "도쿄"): 1100000,
        ("파리", "뉴욕"): 800000,
        ("뉴욕", "서울"): 1500000,
        ("뉴욕", "파리"): 800000,
    }
    
    # 요청된 경로 확인
    route = (depart, arrive)
    
    if route in prices:
        price = prices[route]
        return f"✈️ {depart}에서 {arrive}까지 {date} 항공편 예상 가격은 {format(price, ',')}원입니다."
    else:
        return f"죄송합니다. {depart}에서 {arrive}로 가는 항공편 정보가 없습니다."


# 3. 여행 일정 추천 도구
@tool
def make_itinerary(city: str, days: int) -> str:
    """city에서 days일간 추천 일정을 반환합니다."""
    # 주요 도시별 일정 아이디어
    city_activities = {
        "파리": [
            "에펠탑 방문", "루브르 박물관 관람", "노트르담 대성당 구경", 
            "몽마르트 언덕 산책", "개선문 방문", "세느강 크루즈", 
            "샹젤리제 쇼핑", "베르사유 궁전 투어", "오르세 미술관"
        ],
        "도쿄": [
            "메이지 신궁 방문", "시부야 스크램블 구경", "센소지 사원", 
            "도쿄 타워", "하라주쿠 쇼핑", "긴자 탐방", 
            "아키하바라 전자상가", "스미다강 크루즈", "신주쿠 교엔 공원"
        ],
        "뉴욕": [
            "타임스퀘어 구경", "센트럴 파크 산책", "자유의 여신상 방문", 
            "엠파이어 스테이트 빌딩", "브로드웨이 쇼 관람", "메트로폴리탄 미술관",
            "소호 지역 쇼핑", "브루클린 브릿지 산책", "첼시 마켓 방문"
        ]
    }
    
    # 도시 정보가 있는지 확인
    if city not in city_activities:
        return "해당 도시의 일정 정보가 없습니다."
    
    # 해당 도시의 활동 목록
    activities = city_activities[city]
    
    # 포맷팅된 문자열로 일정 생성
    result = f"## {city} {days}일 추천 여행 일정\n\n"
    
    for day in range(1, days + 1):
        # 아침/점심/저녁 활동 선택 (순환하면서)
        morning = activities[(day * 3 - 3) % len(activities)]
        afternoon = activities[(day * 3 - 2) % len(activities)]
        evening = activities[(day * 3 - 1) % len(activities)]
        
        result += f"### Day {day}\n"
        result += f"- 아침: {morning}\n"
        result += f"- 점심: {city} 현지 레스토랑에서 점심\n"
        result += f"- 오후: {afternoon}\n"
        result += f"- 저녁: {evening} 후 저녁 식사\n\n"
    
    return result

# 4. 최종적으로 사용할 도구 목록
tools = [get_attractions, get_flight_price, make_itinerary]

In [19]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    raise ValueError(".env 파일에 OPENAI_API_KEY가 설정되지 않았습니다.")

print("✅ API 키가 성공적으로 로드되었습니다.")

✅ API 키가 성공적으로 로드되었습니다.


In [20]:
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools, parallel_tool_calls=False)

sys_msg = SystemMessage(content="당신은 여행 계획을 도와주는 여행 전문 에이전트 입니다. 사용자의 요청을 이해하고 적절한 도구를 순차적으로 사용하여 여행 계획을 세워주세요.")

def assistant(state: MessagesState):
    messages = [sys_msg] + state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages" : [response]}

tool_node = ToolNode(tools)

In [21]:
builder = StateGraph(MessagesState)
builder.add_node("assistant", assistant)
builder.add_node("tools", tool_node)

builder.add_edge(START, "assistant")
builder.add_conditional_edges("assistant", tools_condition)
builder.add_edge("tools", "assistant")

graph = builder.compile()



In [22]:
messages = [HumanMessage(content=
    "첫째, 파리의 상위 5개 관광명소를 알려줘. "
    "그다음, 서울→파리 2025-06-01 항공권 예상 가격을 알려주고, "
    "마지막으로 파리 3일 일정 추천해줘."
)]
result = graph.invoke({"messages": messages})

for msg in result["messages"]:
    msg.pretty_print()

================================ Human Message =================================

첫째, 파리의 상위 5개 관광명소를 알려줘. 그다음, 서울→파리 2025-06-01 항공권 예상 가격을 알려주고, 마지막으로 파리 3일 일정 추천해줘.
================================== Ai Message ==================================
Tool Calls:
  get_attractions (call_mpAPeTn9PLZV8d8AWHYqyNp0)
 Call ID: call_mpAPeTn9PLZV8d8AWHYqyNp0
  Args:
    city: 파리
================================= Tool Message =================================
Name: get_attractions

["에펠탑", "루브르박물관", "노트르담 대성당", "몽마르트 언덕", "개선문"]
================================== Ai Message ==================================
Tool Calls:
  get_flight_price (call_LKDyDUqwxRQZHNXpFfWuZGGA)
 Call ID: call_LKDyDUqwxRQZHNXpFfWuZGGA
  Args:
    depart: 서울
    arrive: 파리
    date: 2025-06-01
================================= Tool Message =================================
Name: get_flight_price

✈️ 서울에서 파리까지 2025-06-01 항공편 예상 가격은 1,200,000원입니다.
================================== Ai Message =================================

### 2. 메시지 세이버 적용 전: 상태(State) 누락 문제

In [23]:
messages = HumanMessage(content="내가 몇 일 동안의 파리여행 계획을 짜달라고 했지?")
result = graph.invoke({"messages": [messages]})
for i in result["messages"]:
    i.pretty_print()

================================ Human Message =================================

내가 몇 일 동안의 파리여행 계획을 짜달라고 했지?
================================== Ai Message ==================================

아직 여행 기간에 대한 정보를 받지 못했습니다. 파리 여행을 며칠 동안 계획하시고 싶은지 알려주시면, 그에 맞춰 일정을 짜드리겠습니다.


### 3. 메시지 세이버 적용 후: 상태 누적 및 히스토리 저장

In [24]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
react_graph_memory = builder.compile(checkpointer=memory)

messages = [HumanMessage(content=
                         "첫째, 파리의 상위 5개 관광명소를 알려줘."
                         "그다음, 서울->파리 2025-06-01 항공권 예상 가격을 알려주고, "
                         "마지막으로 파리 3일 일정 추천해줘.")]

config = {"configurable": {"thread_id":"1"}}
result = react_graph_memory.invoke({"messages": messages}, config)
for msg in result["messages"]:
    msg.pretty_print()

================================ Human Message =================================

첫째, 파리의 상위 5개 관광명소를 알려줘.그다음, 서울->파리 2025-06-01 항공권 예상 가격을 알려주고, 마지막으로 파리 3일 일정 추천해줘.
================================== Ai Message ==================================
Tool Calls:
  get_attractions (call_OwC4Eod5D3PDW3fMJ2IQH6af)
 Call ID: call_OwC4Eod5D3PDW3fMJ2IQH6af
  Args:
    city: 파리
================================= Tool Message =================================
Name: get_attractions

["에펠탑", "루브르박물관", "노트르담 대성당", "몽마르트 언덕", "개선문"]
================================== Ai Message ==================================
Tool Calls:
  get_flight_price (call_DSyJ5umEJct7dRrp0qpoF2R9)
 Call ID: call_DSyJ5umEJct7dRrp0qpoF2R9
  Args:
    depart: 서울
    arrive: 파리
    date: 2025-06-01
================================= Tool Message =================================
Name: get_flight_price

✈️ 서울에서 파리까지 2025-06-01 항공편 예상 가격은 1,200,000원입니다.
================================== Ai Message =================================

In [25]:
messages = [HumanMessage(content=
    "내가 이전에 파리여행 계획을 짜달라고 했지?"
)]
result = react_graph_memory.invoke({"messages": messages}, config)
for msg in result["messages"]:
    msg.pretty_print()

================================ Human Message =================================

첫째, 파리의 상위 5개 관광명소를 알려줘.그다음, 서울->파리 2025-06-01 항공권 예상 가격을 알려주고, 마지막으로 파리 3일 일정 추천해줘.
================================== Ai Message ==================================
Tool Calls:
  get_attractions (call_OwC4Eod5D3PDW3fMJ2IQH6af)
 Call ID: call_OwC4Eod5D3PDW3fMJ2IQH6af
  Args:
    city: 파리
================================= Tool Message =================================
Name: get_attractions

["에펠탑", "루브르박물관", "노트르담 대성당", "몽마르트 언덕", "개선문"]
================================== Ai Message ==================================
Tool Calls:
  get_flight_price (call_DSyJ5umEJct7dRrp0qpoF2R9)
 Call ID: call_DSyJ5umEJct7dRrp0qpoF2R9
  Args:
    depart: 서울
    arrive: 파리
    date: 2025-06-01
================================= Tool Message =================================
Name: get_flight_price

✈️ 서울에서 파리까지 2025-06-01 항공편 예상 가격은 1,200,000원입니다.
================================== Ai Message =================================

### 4. 스레드 아이디별 (Thread ID) 대화 세션 관리

In [26]:
memory = MemorySaver()
react_graph_memory = builder.compile(checkpointer=memory)

# 1. 사용자 A 세션 (thread_id="userA")
config_a = {"configurable": {"thread_id": "userA"}}
messages_a1 = [HumanMessage(content="파리 3일 여행 계획 짜줘.")]
result_a1 = react_graph_memory.invoke({"messages": messages_a1}, config_a)

# 2. 사용자 B 세션 (thread_id="userB")
config_b = {"configurable": {"thread_id": "userB"}}
messages_b1 = [HumanMessage(content="도쿄 5일 여행 계획 짜줘.")]
result_b1 = react_graph_memory.invoke({"messages": messages_b1}, config_b)

# 3. 사용자 A가 다시 질문
messages_a2 = [HumanMessage(content="내가 어느 나라로 며칠 동안 여행을 요청했었지?")]
result_a2 = react_graph_memory.invoke({"messages": messages_a2}, config_a)

# 4. 사용자 B도 다시 질문
messages_b2 = [HumanMessage(content="내가 어느 나라로 며칠 동안 여행을 요청했었지?")]
result_b2 = react_graph_memory.invoke({"messages": messages_b2}, config_b)

In [27]:
print("사용자A:", result_a2['messages'][-1].content)
print("사용자B:", result_b2['messages'][-1].content)

사용자A: 당신은 파리로 3일 동안 여행 계획을 요청하셨습니다. 추가적인 정보나 도움이 필요하시면 언제든지 말씀해 주세요!
사용자B: 당신은 도쿄, 일본으로 5일 동안의 여행 계획을 요청하셨습니다. 추가로 도움이 필요하시면 말씀해 주세요!
